# Training a model with PyTorch Lightning

This tutorial provides a quick overview of training a toy model with Lightning, using the `tiledbsoma_ml.ExperimentAxisQueryIterableDataset` class, on data from the [CZI CELLxGENE Census](https://chanzuckerberg.github.io/cellxgene-census/) dataset. This is intended only to demonstrate the use of the `ExperimentAxisQueryIterableDataset`, and not as an example of how to train a biologically useful model.

For more information on these API,  please refer to the [`tutorial_pytorch` notebook](tutorial_pytorch.ipynb).

**Prerequesites**

Install `tiledbsoma_ml` and `scikit-learn`, for example:

> pip install tiledbsoma_ml scikit-learn


## Initialize SOMA Experiment query as training data

In [4]:
import pytorch_lightning as pl
import tiledbsoma_ml as soma_ml
import torch
from sklearn.preprocessing import LabelEncoder

import tiledbsoma as soma

CZI_Census_Homo_Sapiens_URL = "s3://cellxgene-census-public-us-west-2/cell-census/2024-07-01/soma/census_data/homo_sapiens/"

experiment = soma.open(
    CZI_Census_Homo_Sapiens_URL,
    context=soma.SOMATileDBContext(tiledb_config={"vfs.s3.region": "us-west-2"}),
)
obs_value_filter = "tissue_general == 'tongue' and is_primary_data == True"

with experiment.axis_query(
    measurement_name="RNA", obs_query=soma.AxisQuery(value_filter=obs_value_filter)
) as query:
    obs_df = query.obs(column_names=["cell_type"]).concat().to_pandas()
    cell_type_encoder = LabelEncoder().fit(obs_df["cell_type"].unique())

    experiment_dataset = soma_ml.ExperimentAxisQueryIterableDataset(
        query,
        X_name="raw",
        obs_column_names=["cell_type"],
        batch_size=128,
        shuffle=True,
        seed=12345,
    )

## Define the Lightning module

In [5]:
class LogisticRegressionLightning(pl.LightningModule):
    def __init__(self, input_dim, output_dim, cell_type_encoder, learning_rate=1e-5):
        super(LogisticRegressionLightning, self).__init__()
        self.linear = torch.nn.Linear(input_dim, output_dim)
        self.cell_type_encoder = cell_type_encoder
        self.learning_rate = learning_rate
        self.loss_fn = torch.nn.CrossEntropyLoss()

    def forward(self, x):
        outputs = torch.sigmoid(self.linear(x))
        return outputs

    def training_step(self, batch, batch_idx):
        X_batch, y_batch = batch
        # X_batch = X_batch.float()
        X_batch = torch.from_numpy(X_batch).float().to(self.device)

        # Perform prediction
        outputs = self(X_batch)

        # Determine the predicted label
        probabilities = torch.nn.functional.softmax(outputs, 1)
        predictions = torch.argmax(probabilities, axis=1)

        # Compute loss
        # y_batch = y_batch.flatten()
        y_batch = torch.from_numpy(
            self.cell_type_encoder.transform(y_batch["cell_type"])
        ).to(self.device)
        loss = self.loss_fn(outputs, y_batch.long())

        # Compute accuracy
        train_correct = (predictions == y_batch).sum().item()
        train_accuracy = train_correct / len(predictions)

        # Log loss and accuracy
        self.log("train_loss", loss, prog_bar=True)
        self.log("train_accuracy", train_accuracy, prog_bar=True)

        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer

## Train the model

In [6]:
dataloader = soma_ml.experiment_dataloader(
    experiment_dataset, num_workers=2, persistent_workers=True
)

# The size of the input dimension is the number of genes
input_dim = experiment_dataset.shape[1]

# The size of the output dimension is the number of distinct cell_type values
output_dim = len(cell_type_encoder.classes_)

# Initialize the PyTorch Lightning model
model = LogisticRegressionLightning(
    input_dim, output_dim, cell_type_encoder=cell_type_encoder
)

# Define the PyTorch Lightning Trainer
trainer = pl.Trainer(
    max_epochs=20,
    # accelerator=args.accelerator,
    # strategy="ddp",
)

# set precision
torch.set_float32_matmul_precision("high")

# Train the model
trainer.fit(model, train_dataloaders=dataloader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type             | Params | Mode 
-----------------------------------------------------
0 | linear  | Linear           | 726 K  | train
1 | loss_fn | CrossEntropyLoss | 0      | train
-----------------------------------------------------
726 K     Trainable params
0         Non-trainable params
726 K     Total params
2.905     Total estimated model params size (MB)
2         Modules in train mode
0         Modules in eval mode
/home/ubuntu/miniforge3/envs/toymodel/lib/python3.11/site-packages/pytorch_lightning/utilities/data.py:122: Your `IterableDataset` has `__len__` defined. In combination with multi-process data loading (when num_workers > 1), `__len__` could be inaccurate if each worker is not configured independently to avoid having duplicate data.
/home/ubuntu/miniforge3/envs/toymodel/lib/python3.11/site-pack

Epoch 19: 100%|██████████| 118/118 [00:17<00:00,  6.87it/s, v_num=6, train_loss=1.680, train_accuracy=0.977]

`Trainer.fit` stopped: `max_epochs=20` reached.


Epoch 19: 100%|██████████| 118/118 [00:17<00:00,  6.86it/s, v_num=6, train_loss=1.680, train_accuracy=0.977]
